In [22]:
# Dependencies
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras import Input

In [21]:
# Load data (FRED-MD)
df = pd.read_csv('../current.csv')

# Remove the first row (transformation codes)
transformation_codes = df.iloc[0]  # Transformation codes can be applied if needed
df = df.iloc[1:]

# Set the first column as the index and datetime
df.set_index(df.columns[0], inplace=True)
df.index = pd.to_datetime(df.index)

# Dropna
data = df.dropna()

# Create train data and target
target = (data['CPIAUCSL'].diff(12) / data['CPIAUCSL'].shift(12)) * 100
target = target.shift(-12).dropna()
data = data.loc[target.index]
train = data.dropna()

In [ ]:
# Generate RMSE for a variety of horizons
for forecast_horizon in range(4, 12 * 5 + 4, 12):
    # Meta parameters
    context_window = 4 # 1 quarter
    cutoff = len(train) - forecast_horizon # pretend we are at this point in time
    dropout_rate = 0

    # Create training data
    X, y = [], []
    for i in range(cutoff - context_window):
        context = train.iloc[i:i + context_window].values
        future = target.iloc[i + context_window : i + context_window + forecast_horizon].values
        if len(future) == forecast_horizon:  # avoid short sequences at the end
            X.append(context)
            y.append(future)
    X, y = np.array(X), np.array(y)
    
    # Normalize the data
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).reshape(y.shape)
    X_train, y_train = X_scaled, y_scaled

    # Build LSTM model
    model = Sequential()
    # Input layer
    model.add(Input(shape=(context_window, X.shape[-1])))
    # First LSTM layer
    model.add(LSTM(units=64, return_sequences=True))
    model.add(Dropout(dropout_rate))
    # Second LSTM layer
    model.add(LSTM(units=64, return_sequences=False))
    model.add(Dropout(dropout_rate))
    # Output layer for forecast_horizon steps
    model.add(Dense(units=forecast_horizon))
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Training the model
    model.fit(
        X_train, y_train, 
        epochs=50, 
        batch_size=32,
        verbose=0
    )

    # Get data for prediction
    context = train.iloc[-forecast_horizon - context_window : -forecast_horizon].values
    context_scaled = scaler_X.transform(context).reshape(1, context_window, -1)
    # Forecast
    y_pred_scaled = model.predict(context_scaled)
    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    # Print RMSE
    rmse = np.sqrt(mean_squared_error(target.iloc[-forecast_horizon:], y_pred))
    print(f'Forecast horizon: {forecast_horizon}, RMSE: {rmse:.2f}')
    with open('rmse.txt', 'a') as f:
        f.write(f'Forecast horizon: {forecast_horizon}, RMSE: {rmse:.2f}, Context: {context_window}, Dropout: {dropout_rate}\n')
        f.write(f'{y_pred}\n{target.iloc[-forecast_horizon:].values}\n\n')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
Forecast horizon: 4, RMSE: 0.42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
Forecast horizon: 16, RMSE: 1.22
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
Forecast horizon: 28, RMSE: 0.81
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
Forecast horizon: 40, RMSE: 1.32
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step
Forecast horizon: 52, RMSE: 2.00


Old Architecture:
Forecast horizon: 4, RMSE: 0.19
Forecast horizon: 16, RMSE: 0.61
Forecast horizon: 28, RMSE: 0.85
Forecast horizon: 40, RMSE: 1.17
Forecast horizon: 52, RMSE: 2.05